In [1]:
# Good explaination of how to think of rolling mean
# https://www.portent.com/blog/analytics/rolling-averages-math-moron.htm

In [2]:
import tensorflow as tf
import numpy as np

In [9]:
# Explicitly create a Graph object
graph = tf.Graph()

with graph.as_default():
    
    with tf.name_scope("variables"):
        # Variable to keep track of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, name="global_step")
        
        # Variable that keeps track of the sum of all output values over time:
        total_output = tf.Variable(0.0, dtype=tf.float32, name="total_output")
        
        avgs = tf.Variable(0, dtype=tf.float32, name="avgs")
    
    # Primary transformation Operations
    with tf.name_scope("transformation"):
        
        # Separate input layer
        with tf.name_scope("input"):
            # Create input placeholder- takes in a Vector 
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
    
        # Separate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")

        # Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
        
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        update_total = total_output.assign_add(output)
        
        # Increments the above `global_step` Variable, should be run whenever the graph is run
        increment_step = global_step.assign_add(1)
        
    # Summary Operations
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        avgs = tf.stack([avgs, avg], axis=0)
        total_avgs = tf.reduce_sum(avgs, name="total_avgs")
        rolling_mean = total_avgs / tf.cast(increment_step, tf.float32)
        
        # Creates summaries for output node
        tf.summary.scalar('Output', output)
        tf.summary.scalar('Sum_of_outputs_over_time', update_total) 
        tf.summary.scalar('Average_of_outputs_over_time', avg)
        tf.summary.scalar('rolling_mean_of_outputs_over_time', rolling_mean)
#         tf.summary.scalar('max_number_over_time', update_max) 
    
    # Global Variables and Operations
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.global_variables_initializer()    
        # Merge all summaries into one Operation
        merged_summaries = tf.summary.merge_all()

# Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)

# Open a SummaryWriter to save summaries
writer = tf.summary.FileWriter('problem-1', graph)

# Initialize Variables
sess.run(init)

In [10]:
def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}
    out, step, summary, rolling_meany, so_avg = sess.run([output, increment_step, merged_summaries, rolling_mean, avg], feed_dict=feed_dict)
    print("avg: {0}, rolling_mean: {1}".format(so_avg, rolling_meany))
    writer.add_summary(summary, global_step=step)

In [11]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

avg: 26.0, rolling_mean: 26.0
avg: 31.5, rolling_mean: 15.75
avg: 26.3333339691, rolling_mean: 8.77777767181
avg: 22.75, rolling_mean: 5.6875
avg: 30.0, rolling_mean: 6.0
avg: 26.5, rolling_mean: 4.41666650772
avg: 27.2857151031, rolling_mean: 3.89795923233
avg: 27.25, rolling_mean: 3.40625
avg: 24.4444446564, rolling_mean: 2.71604943275
avg: 35.5, rolling_mean: 3.54999995232


In [6]:
# Write the summaries to disk
writer.flush()

In [7]:
# Close the tf.summary.FileWriter
writer.close()

In [8]:
# Close the session
sess.close()

To start TensorBoard after running this code, run the following command:

```
$ tensorboard --logdir 'problem-1'
```